## 📚 Exercise 13: Entity & Relation Extraction

### Task 1: Relation extraction from Wikipedia articles

Use Wikipedia to extract the relation `directedBy(Movie, Person)` by applying pattern based heuristics that utilize: *Part Of Speech Tagging*, *Named Entity Recognition* and *Regular Expressions*.

#### Required Library: SpaCy
- ```conda install -y spacy```
- ```python -m spacy download en```

In [1]:
import urllib.request, json, csv, re
import spacy
nlp = spacy.load('en_core_web_sm')

In [2]:
#read tsv with input movies
def read_tsv():
    movies=[]
    with open('movies.tsv','r') as file:
        tsv = csv.reader(file, delimiter='\t')
        next(tsv) #remove header
        movies = [{'movie':line[0], 'director':line[1]} for line in tsv]
    return movies

#parse wikipedia page
def parse_wikipedia(movie):
    txt = ''
    try:
        with urllib.request.urlopen('https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro=&explaintext=&titles='+movie) as url:
            data = json.loads(url.read().decode())
            txt = next (iter (data['query']['pages'].values()))['extract']
    except:
        pass
    return txt

#### 1) Parse the raw text of a Wikipedia movie page and extract named (PER) entities.

In [8]:
def find_PER_entities(txt):
    persons = []
    doc = nlp(txt)

    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            persons.append(ent.text)
    return persons

#### 2) Given the raw text of a Wikipedia movie page and the extracted PER entities, find the director.

In [28]:
def find_director(txt, persons):
    txt = re.sub("[!?,.]", "", txt).split()
    for p1 in range(0, len(txt)):
        if txt[p1] == "directed":
            for p2 in range(p1, len(txt)):
                for per in persons:
                    if per.startswith(txt[p2]):
                        return per
    return ""

In [33]:
movies = read_tsv()

In [38]:
statements=[]
directors = []
for m in movies:
    txt = parse_wikipedia(m['movie'])
    persons = find_PER_entities(txt)
    director = find_director(txt, persons)
    directors.append(director)
    
    if director != '':
        statements.append(m['movie'] + ' is directed by ' + director + '.')

#### 3) Compute the precision and recall based on the given ground truth (column Director from tsv file) and show examples of statements that are extracted.

In [53]:
directors_gt = [m['director'] for m in movies]

In [55]:
# compute precision and recall
precision = len(set(directors) & set(directors_gt)) / len(statements)
recall = len(set(directors) & set(directors_gt)) / len(directors_gt)
print ('Precision:',precision)
print ('Recall:',recall)
print('\n***Sample Statements***')
print(statements[:5])

Precision: 0.6141078838174274
Recall: 0.5156794425087108

***Sample Statements***
['13_Assassins_(2010_film) is directed by Takashi Miike.', '14_Blades is directed by Daniel Lee.', '22_Bullets is directed by Richard Berry.', 'Alien_vs_Ninja is directed by Seiji Chiba.', 'Bad_Blood_(2010_film) is directed by Dennis Law.']


## Task 2: Named Entity Recognition using Hidden Markov Model


Define a Hidden Markov Model (HMM) that recognizes Person (*PER*) entities.
Particularly, your model must be able to recognize pairs of the form (*firstname lastname*) as *PER* entities.
Using the given sentences as training and test set:

In [56]:
training_set=['The best blues singer was Bobby Bland while Ray Charles pioneered soul music .', \
              'Bobby Bland was just a singer whereas Ray Charles was a pianist , songwriter and singer .' \
              'None of them lived in Chicago .']

test_set=['Ray Charles was born in 1930 .', \
          'Bobby Bland was born the same year as Ray Charles .', \
          'Muddy Waters is the father of Chicago Blues .']

#### 1) Annotate your training set with the labels I (for PER entities) and O (for non PER entities).
	
    *Hint*: Represent the sentences as sequences of bigrams, and label each bigram.
	Only bigrams that contain pairs of the form (*firstname lastname*) are considered as *PER* entities.

In [58]:
#Bigram Representation
def getBigrams(sents):
    return [b[0]+' '+b[1] for l in sents for b in zip(l.split(' ')[:-1], l.split(' ')[1:])]

bigrams = getBigrams(training_set)

#Annotation
PER = ['Bobby Bland', 'Ray Charles']
annotations = [[[b, 1 if b in PER else 0] for b in bigrams] for s in training_set]
print('Annotation\n', annotations,'\n')

Annotation
 [[['The best', 0], ['best blues', 0], ['blues singer', 0], ['singer was', 0], ['was Bobby', 0], ['Bobby Bland', 1], ['Bland while', 0], ['while Ray', 0], ['Ray Charles', 1], ['Charles pioneered', 0], ['pioneered soul', 0], ['soul music', 0], ['music .', 0], ['Bobby Bland', 1], ['Bland was', 0], ['was just', 0], ['just a', 0], ['a singer', 0], ['singer whereas', 0], ['whereas Ray', 0], ['Ray Charles', 1], ['Charles was', 0], ['was a', 0], ['a pianist', 0], ['pianist ,', 0], [', songwriter', 0], ['songwriter and', 0], ['and singer', 0], ['singer .None', 0], ['.None of', 0], ['of them', 0], ['them lived', 0], ['lived in', 0], ['in Chicago', 0], ['Chicago .', 0]], [['The best', 0], ['best blues', 0], ['blues singer', 0], ['singer was', 0], ['was Bobby', 0], ['Bobby Bland', 1], ['Bland while', 0], ['while Ray', 0], ['Ray Charles', 1], ['Charles pioneered', 0], ['pioneered soul', 0], ['soul music', 0], ['music .', 0], ['Bobby Bland', 1], ['Bland was', 0], ['was just', 0], ['just 

#### 2) Compute the transition and emission probabilities for the HMM (use smoothing parameter $\lambda$=0.5).

    *Hint*: For the emission probabilities you can utilize the morphology of the words that constitute a bigram (e.g., you can count their uppercase first characters).

In [ ]:
lambda_ = 0.5

#Transition Probabilities
transition_prob={}


#Prior
transition_prob['P(I|start)'] = ...
transition_prob['P(O|start)'] = ...

transition_prob['P(O|O)'] = ...
transition_prob['P(O|I)'] = ...
transition_prob['P(I|O)'] = ...
transition_prob['P(I|I)'] = ...

print('Transition Probabilities\n',transition_prob, '\n')

#Emission Probabilities
emission_prob={}

        
default_emission = ...

emission_prob['P(2_upper|O)'] = ...
emission_prob['P(2_upper|I)'] = ...
emission_prob['P(1_upper|O)'] = ...
emission_prob['P(1_upper|I)'] = ...
emission_prob['P(0_upper|O)'] = ...
emission_prob['P(0_upper|I)'] = ...

print('Emission Probabilities\n', emission_prob, '\n')

#### 3) Predict the labels of the test set and compute the precision and the recall of your model.

In [ ]:
#Prediction
bigrams = getBigrams(test_set)
entities=[]
prev_state='start'
for b in bigrams:
    I_prob = ...
    O_prob = ...
    
    if ...:
        prev_state = 'O'
    else:
        entities.append(b)
        prev_state = 'I'

print('Predicted Entities\n', entities, '\n')

Precision is *...%* while recall is *...%*. 

#### 4) Comment on how you can further improve this model.

...